In [ ]:
# main.py

def extract_start_end(text):
    text = text.lower()
    # English
        "lib": "Library",
        "library": "Library",
        "eng": "Engineering Building",
        "engineering": "Engineering Building",
        "engineering building": "Engineering Building",
        "caf": "Cafeteria",
        "cafeteria": "Cafeteria",
        "admin": "Administration",
        "administration": "Administration",
        "gym": "Gym",
        "hall": "Hall",
        "res": "Residence",
        "residence": "Residence",
        "dorm": "Residence",
        "amphi": "Amphitheater",
        "amphitheater": "Amphitheater",
        "lab": "Laboratory",
        "laboratory": "Laboratory",
        "labo": "Laboratory",

        # French
        "biblio": "Library",
        "bibliothèque": "Library",
        "fac": "Engineering Building",
        "faculté": "Engineering Building",
        "département": "Engineering Building",
        "cantine": "Cafeteria",
        "salle de sport": "Gym",
        "gymnase": "Gym",
        "auditorium": "Auditorium",
        "salle": "Hall",
        "amphithéâtre": "Amphitheater",

        # Tunisian dialect / Arabic
        "maktaba": "Library",
        "el-maktaba": "Library",
        "qahwa": "Cafeteria",
        "win el qahwa": "Cafeteria",
        "el-cafeteria": "Cafeteria",
        "kulia": "Engineering Building",
        "departement": "Engineering Building",
        "mabna al-handasa": "Engineering Building",
        "el-gym": "Gym",
        "el-amphi": "Amphitheater",
        "maamoura": "Residence",
        "ma3moura": "Residence"
    }

    dialect_mapping = {
        # Tunisian / Arabic words
        "وين": "where",
        "كيفاش": "how",
        "يمشي": "go",
        "imchi": "go",
        "min": "from",
        "ila": "to",
        "win": "where",
        "kifesh": "how",
        "yani": "meaning",

        # French prepositions
        "de": "from",
        "à": "to",
        "vers": "to",

        # English variants
        "from": "from",
        "to": "to",
        "men": "from",
        "lel": "to",
        "lil": "to"
    }
    for k, v in dialect_mapping.items():
        text = text.replace(k, v)

    found = [key for key in buildings if key in text]

    # NEW CASE: Same location repeated (e.g. "from cafeteria to cafeteria")
    if len(found) == 1 and text.count(found[0]) > 1:
        location = buildings[found[0]]
        return location, location

    if len(found) >= 2:

        # Ensure correct start and end detection based on "from" and "to"
        if 'from' in text and 'to' in text:
 
            if text.find('from') < text.find('to'):
                
                start, end = found[0], found[1]
            else:
                start, end = found[1], found[0]
        elif 'from' in text:
            start, end = found[0], found[1]
        elif 'to' in text:
            start, end = found[0], found[1]
        else:
            # fallback to order in text
            start, end = found[0], found[1]

        return buildings[start], buildings[end]

    if len(found) == 1:
        current = buildings[found[0]]
        raise ValueError(f"You mentioned only one location: {current}. Where are you going?")

    raise ValueError("Could not find matching locations.")



def get_navigation_instruction(start, end, matrix):
    if start == end:
        return f"🚩 You're already at {start}."

    if matrix.get(start, {}).get(end) is not None:
        distance = matrix[start][end]
        return f"✅ Direct route: {start} is within {distance} units of {end}."

    for mid in matrix:
        if mid != start and mid != end:
            if matrix.get(start, {}).get(mid) is not None and matrix.get(mid, {}).get(end) is not None:
                d1 = matrix[start][mid]
                d2 = matrix[mid][end]
                total = round(d1 + d2, 2)
                return (f"🔁 No direct route from {start} to {end}.\n"
                        f"➡️ Go from {start} to {mid} ({d1} units), "
                        f"then from {mid} to {end} ({d2} units).\n"
                        f"🧮 Total distance: {total} units.")

    return f"❌ No available path found from {start} to {end}."
# bot.py


def ask_bot(user_input):
    try:
        start, end = extract_start_end(user_input)
        return get_navigation_instruction(start, end, distance_matrix)
    except Exception as e:
        return f"❓ Sorry, I couldn't understand. Please specify buildings. ({str(e)})"


# distance.py
distance_matrix = {
    "Library": {
        "Engineering Building": 4.47,
        "Cafeteria": 3.61,
    },
    "Engineering Building": {
        "Library": 4.47,
    },
    "Cafeteria": {
        "Library": 3.61,
    }
}




# parser.py

def test_mode():
    test_inputs = [
        "win imchi lil maktaba?",
        "kifesh  men engineering lel library?",
        "How to go from engineering to library?",
        "maktaba finha?",
        "Kif nemchi lil kulia?",
        "nimchi men engineering lel maktaba"
    ]

    print("🔍 Running Test Cases...\n")
    for i, inp in enumerate(test_inputs):
        print(f"\n🧪 Test Case {i+1}: \"{inp}\"")
        response = ask_bot(inp)
        print(f"Bot: {response}")
        print("-" * 40)

if __name__ == "__main__":
    print("🔵 Smart Campus Navigation Bot")
    mode = input("Choose mode [1: Interactive, 2: Test]: ").strip()

    if mode == "2":
        test_mode()
    else:
        print("Type 'exit' to quit.\n")
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                print("👋 Goodbye!")
                break
            response = ask_bot(user_input)
            print(f"Bot: {response}")

🔵 Smart Campus Navigation Bot
🔍 Running Test Cases...


🧪 Test Case 1: "win imchi lil maktaba?"
Bot: ❓ Sorry, I couldn't understand. Please specify buildings. (You mentioned only one location: Library. Where are you going?)
----------------------------------------

🧪 Test Case 2: "kifesh  men engineering lel library?"
Bot: ✅ Direct route: Library is within 4.47 units of Engineering Building.
----------------------------------------

🧪 Test Case 3: "How to go from engineering to library?"
Bot: ✅ Direct route: Engineering Building is within 4.47 units of Library.
----------------------------------------

🧪 Test Case 4: "maktaba finha?"
Bot: ❓ Sorry, I couldn't understand. Please specify buildings. (You mentioned only one location: Library. Where are you going?)
----------------------------------------

🧪 Test Case 5: "Kif nemchi lil kulia?"
Bot: ❓ Sorry, I couldn't understand. Please specify buildings. (You mentioned only one location: Engineering Building. Where are you going?)
--------